In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

In [83]:
path = "./data/"
file2022 = "flightlist_20221001_20221031.csv"
file2019 = "flightlist_20190101_20190131.csv.gz"
usecols = ["callsign", "number", "icao24", "origin", "destination", "day"]
database2022 = pd.read_csv(path + file2022, usecols=usecols, parse_dates=["day"])
database2019 = pd.read_csv(path + file2019, usecols = usecols, compression="gzip", parse_dates=["day"])

/tmp/ipykernel_5007/3116359885.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  database2019 = pd.read_csv(path + file2019, usecols = usecols, compression="gzip", parse_dates=["day"])


In [4]:
database2022.head()

,callsign,number,icao24,origin,destination,day
0,QQE894,NaN,06a1cb,NaN,DAAG,2022-10-01 00:00:00+00:00
1,WJA1417,WS1417,c07eee,CYYZ,CYYC,2022-10-01 00:00:00+00:00
2,KZR1377,NaN,4cac0f,RKSI,LTBA,2022-10-01 00:00:00+00:00
3,TGW842,NaN,76bd4a,YSSY,RKSI,2022-10-01 00:00:00+00:00
4,BAW16,NaN,406f74,YSSY,EGLL,2022-10-01 00:00:00+00:00


In [5]:
database2019.head()

,callsign,number,icao24,origin,destination,day
0,HVN19,NaN,888152,YMML,LFPG,2019-01-01 00:00:00+00:00
1,CCA839,NaN,780ad1,YMML,LEBL,2019-01-01 00:00:00+00:00
2,CES219,NaN,780b7e,YSSY,EDDF,2019-01-01 00:00:00+00:00
3,AEA040,NaN,34444e,LEMD,LEMD,2019-01-01 00:00:00+00:00
4,CXA825,NaN,780d75,YSSY,LFPG,2019-01-01 00:00:00+00:00


In [11]:
database = [database2019, database2022]

In [15]:
database2022.describe(datetime_is_numeric=True)

,callsign,number,icao24,origin,destination,day
count,3091445,651981,3091425,2428315,2615384,3091445
unique,273939,38223,122356,11680,13343,NaN
top,00000000,WN3193,34610b,KATL,KATL,NaN
freq,1717,128,346,29393,29971,NaN
mean,NaN,NaN,NaN,NaN,NaN,2022-10-15 13:18:41.397533696+00:00
min,NaN,NaN,NaN,NaN,NaN,2022-10-01 00:00:00+00:00
25%,NaN,NaN,NaN,NaN,NaN,2022-10-08 00:00:00+00:00
50%,NaN,NaN,NaN,NaN,NaN,2022-10-15 00:00:00+00:00
75%,NaN,NaN,NaN,NaN,NaN,2022-10-23 00:00:00+00:00
max,NaN,NaN,NaN,NaN,NaN,2022-10-31 00:00:00+00:00


In [6]:
database2022.shape

(3091445, 6)

# Import Airports Database

In [54]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def country_name_to_continent_code(country_name):
    try:
        country_code = country_name_to_country_alpha2(country_name)
        continent = country_alpha2_to_continent_code(country_code)
        return continent
    except:
        return np.nan   

In [81]:
fileAirports = "airports.csv"
dtypes = {"Name": str, "City": str, "Country": str, "IATA": str, "ICAO": str, "Latitude": np.float32, "Longitude": np.float32}
usecols = list(dtypes.keys())
airports = pd.read_csv(path + fileAirports, usecols=usecols, dtype = dtypes, na_values="\\N")
airports["Continent"] = airports.Country.apply(country_name_to_continent_code)
airports.rename(columns = {"Name": "Airport_Name"}, inplace = True)

In [61]:
airports.head()

,Airport_Name,City,Country,IATA,ICAO,Latitude,Longitude,Continent
0,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,OC
1,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,OC
2,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,OC
3,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725983,OC
4,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,OC


# Join the datasets

In [84]:
for dir in ["origin", "destination"]:
    database2022 = database2022.merge(airports.add_prefix(dir + "_"), 
                        how = "left", left_on = dir, right_on = dir + "_ICAO", validate = "m:1")

In [85]:
database2022.head()

,callsign,number,icao24,origin,destination,day,origin_Airport_Name,origin_City,origin_Country,origin_IATA,...,origin_Longitude,origin_Continent,destination_Airport_Name,destination_City,destination_Country,destination_IATA,destination_ICAO,destination_Latitude,destination_Longitude,destination_Continent
0,QQE894,NaN,06a1cb,NaN,DAAG,2022-10-01 00:00:00+00:00,Sun Island Resort and SPA,South Aari Atoll,Maldives,NaN,...,72.862991,AS,Houari Boumediene Airport,Algier,Algeria,ALG,DAAG,36.691002,3.215410,AF
1,WJA1417,WS1417,c07eee,CYYZ,CYYC,2022-10-01 00:00:00+00:00,Lester B. Pearson International Airport,Toronto,Canada,YYZ,...,-79.630600,NA,Calgary International Airport,Calgary,Canada,YYC,CYYC,51.113899,-114.019997,NA
2,KZR1377,NaN,4cac0f,RKSI,LTBA,2022-10-01 00:00:00+00:00,Incheon International Airport,Seoul,South Korea,ICN,...,126.450996,AS,Atatürk International Airport,Istanbul,Turkey,ISL,LTBA,40.976898,28.814600,AS
3,TGW842,NaN,76bd4a,YSSY,RKSI,2022-10-01 00:00:00+00:00,Sydney Kingsford Smith International Airport,Sydney,Australia,SYD,...,151.177002,OC,Incheon International Airport,Seoul,South Korea,ICN,RKSI,37.469101,126.450996,AS
4,BAW16,NaN,406f74,YSSY,EGLL,2022-10-01 00:00:00+00:00,Sydney Kingsford Smith International Airport,Sydney,Australia,SYD,...,151.177002,OC,London Heathrow Airport,London,United Kingdom,LHR,EGLL,51.470600,-0.461941,EU


In [87]:
database2022.shape

(3091445, 22)

In [88]:
database2022.dropna(subset=["origin_IATA", "destination_IATA"], inplace=True)
database2022.shape

(1428223, 22)

In [89]:
database2022 = database2022.loc[database2022["origin"] != database2022["destination"]]
database2022.shape

(1249370, 22)

In [98]:
database2022 = database2022.loc[(database2022.origin_Continent == "EU") | (database2022.destination_Continent == "EU")]

In [104]:
df = database2022.groupby(["origin_IATA", "destination_IATA"]).nunique().reset_index()

In [107]:
df.loc[df.origin_IATA == "VIE"]

,origin_IATA,destination_IATA,callsign,number,icao24,origin,destination,day,origin_Airport_Name,origin_City,...,origin_Latitude,origin_Longitude,origin_Continent,destination_Airport_Name,destination_City,destination_Country,destination_ICAO,destination_Latitude,destination_Longitude,destination_Continent
27101,VIE,ACH,8,1,5,1,1,23,1,1,...,1,1,1,1,1,1,1,1,1,1
27102,VIE,ADJ,1,1,3,1,1,3,1,1,...,1,1,1,1,1,1,1,1,1,1
27103,VIE,AGP,9,1,43,1,1,30,1,1,...,1,1,1,1,1,1,1,1,1,1
27104,VIE,AHO,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
27105,VIE,ALC,4,0,15,1,1,15,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27302,VIE,YUL,1,1,3,1,1,19,1,1,...,1,1,1,1,1,1,1,1,1,1
27303,VIE,YYZ,1,1,14,1,1,30,1,1,...,1,1,1,1,1,1,1,1,1,1
27304,VIE,ZAG,2,0,2,1,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1
27305,VIE,ZAZ,2,0,7,1,1,9,1,1,...,1,1,1,1,1,1,1,1,1,1
